In [12]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from sklearn.metrics import classification_report



from keras.preprocessing.image import ImageDataGenerator

import subprocess

In [13]:
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('data_generation/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('data_generation/testing',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 500,
                         epochs = 3,
                         validation_data = test_set,
                         validation_steps = 50)

Found 16000 images belonging to 2 classes.
Found 4800 images belonging to 2 classes.
Epoch 1/3
500/500 [==============================] - 56s 111ms/step - loss: 0.4242 - acc: 0.7715 - val_loss: 0.2255 - val_acc: 0.8700
Epoch 2/3
500/500 [==============================] - 52s 104ms/step - loss: 0.2778 - acc: 0.8615 - val_loss: 0.1900 - val_acc: 0.9137
Epoch 3/3
500/500 [==============================] - 48s 97ms/step - loss: 0.2378 - acc: 0.8874 - val_loss: 0.1636 - val_acc: 0.9337


In [22]:
import numpy 
test_steps_per_epoch = numpy.math.ceil(test_set.samples / test_set.batch_size)
predictions = classifier.predict_generator(test_set, steps=test_steps_per_epoch)
predicted_classes = numpy.argmax(predictions, axis=1)
true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys())

In [23]:
from sklearn.metrics import confusion_matrix
print("Classification Report: ")
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)
print("Confusion Matrix")
report = confusion_matrix(true_classes, predicted_classes)
print(report)

Classification Report: 
                  precision    recall  f1-score   support

   false_samples       0.58      1.00      0.74      2800
positive_samples       0.00      0.00      0.00      2000

     avg / total       0.34      0.58      0.43      4800

Confusion Matrix
[[2800    0]
 [2000    0]]


/Users/benbrooks/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
